In [69]:
import pandas as pd
import numpy as np
import s3fs
import boto3
import os

In [62]:
desired_img = pd.read_csv('images_to_label.csv', index_col=0, names = ["label","grp"])

In [63]:
desired_img.head()

,label,grp
ID,Likely Label,Assigned to
877,missing ramp,A
1042,missing ramp,A
1205,missing ramp,A
3613,missing ramp,A


In [14]:
set_imgs = set(desired_img.index)

In [22]:
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/gsv/')
fnames2 = fs.ls('s3://streetview-w210/')

In [36]:
to_pull = []
for x in fnames:
    if x.endswith('.json'):
        x_id = np.int64(x[20:-5])
        if x_id in set_imgs:
            for i in ['_45','_135','_225','_315']:
                fname = x[:-5] + i + ".jpg"
                to_pull.append(fname)
for x in fnames2:
    if x.endswith('.json'):
        x_id = np.int64(x[16:-5])
        if x_id in set_imgs:
            for i in ['_45','_135','_225','_315']:
                fname = x[:-5] + i + ".jpg"
                to_pull.append(fname)

In [37]:
len(to_pull)

2248

In [70]:
s3 = boto3.resource('s3') 
bucket = s3.Bucket('streetview-w210')
for fname in to_pull:
    src = fname[16:]
    if src.startswith('gsv/'):
        x = src[4:].split("_")
        grp = desired_img.grp.loc[x[0]]
        dest = "images/" + grp + "/" + src[4:]
    else:
        x = src.split("_")
        grp = desired_img.grp.loc[x[0]]
        dest = "images/" + grp + "/" + src
    bucket.download_file(src, dest)

Use terminal to convert to zip:

`zip -r -X A.zip A`